<a href="https://colab.research.google.com/github/RyanTokManMokMTM/NLP_Training5/blob/main/translation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np

batch_size = 64  # Batch size for training.
epochs = 100  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 10000  # Number of samples to train on.
# Path to the data txt file on disk.
data_path = 'engChinese.txt'

# Vectorize the data.
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
with open(data_path, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text = line.split('\t')
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

# mapping token to index， easily to vectors
input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])


# np.zeros(shape, dtype, order)
# shape is an tuple, in here 3D
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')


# input_texts contain all english sentences
# output_texts contain all chinese sentences
# zip('ABC','xyz') ==> Ax By Cz, looks like that
# the aim is: vectorilize text, 3D
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        # 3D vector only z-index has char its value equals 1.0
        encoder_input_data[i, t, input_token_index[char]] = 1.
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            # igone t=0 and start t=1, means 
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.

# ===========above: encoder-decoder model================


# Define an input sequence and process it.
# input prodocts keras tensor, to fit keras model!
# 1x73 vector 
# encoder_inputs is a 1x73 tensor!
encoder_inputs = Input(shape=(None, num_encoder_tokens))

# units=256, return the last state in addition to the output
encoder_lstm = LSTM((latent_dim), return_state=True)

# LSTM(tensor) return output, state-history, state-current
encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs)

# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# ============above: one RNN/LSTM input to output and state==============


# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))

# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM((latent_dim), return_sequences=True, return_state=True)

# obtain output
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,initial_state=encoder_states)

# ============above: another RNN/LSTM decoder parts====================

# dense 2580x1 units full connented layer
decoder_dense = Dense(num_decoder_tokens, activation='softmax')

# why let decoder_outputs go through dense ?
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn, groups layers into an object 
# with training and inference features
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
# model(input, output)
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Run training
# compile -> configure model for training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
# model optimizsm
model.fit([encoder_input_data, decoder_input_data], 
          decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)
# Save model
model.save('seq2seq.h5')

# Next: inference mode (sampling).
# Here's the drill:
# 1) encode input and retrieve initial decoder state
# 2) run one step of decoder with this initial state
# and a "start of sequence" token as target.
# Output will be the next target token
# 3) Repeat with the current target token and current states

# Define sampling models
encoder_model = Model(encoder_inputs, encoder_states)
# tensor 73x1
decoder_state_input_h = Input(shape=(latent_dim,))
# tensor 73x1
decoder_state_input_c = Input(shape=(latent_dim,))
# tensor 146x1
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
# lstm
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
# 
decoder_states = [state_h, state_c]
# 
decoder_outputs = decoder_dense(decoder_outputs)
# 
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())


def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.
    # this target_seq you can treat as initial state

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        # argmax: Returns the indices of the maximum values along an axis
        # just like find the most possible char
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        # find char using index
        sampled_char = reverse_target_char_index[sampled_token_index]
        # and append sentence
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        # append then ?
        # creating another new target_seq
        # and this time assume sampled_token_index to 1.0
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        # update states, frome the front parts
        states_value = [h, c]

    return decoded_sentence


for seq_index in range(100,200):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

Number of samples: 4000
Number of unique input tokens: 86
Number of unique output tokens: 3023
Max sequence length for inputs: 50
Max sequence length for outputs: 42
Epoch 1/100
50/50 [==============================] - 15s 144ms/step - loss: 2.2732 - val_loss: 2.2418
Epoch 2/100
50/50 [==============================] - 5s 104ms/step - loss: 2.2120 - val_loss: 2.2433
Epoch 3/100
50/50 [==============================] - 5s 105ms/step - loss: 2.2050 - val_loss: 2.2376
Epoch 4/100
50/50 [==============================] - 5s 105ms/step - loss: 2.2016 - val_loss: 2.2392
Epoch 5/100
50/50 [==============================] - 6s 111ms/step - loss: 2.1947 - val_loss: 2.2468
Epoch 6/100
50/50 [==============================] - 5s 105ms/step - loss: 2.1920 - val_loss: 2.2437
Epoch 7/100
50/50 [==============================] - 5s 105ms/step - loss: 2.1916 - val_loss: 2.2436
Epoch 8/100
50/50 [==============================] - 5s 106ms/step - loss: 2.1890 - val_loss: 2.2472
Epoch 9/100
50/50 [======

In [77]:
len(target_characters)

3866

In [1]:
#install package
!pip install opencc

     |████████████████████████████████| 765 kB 4.3 MB/s 


In [2]:
from google.colab import drive
drive.mount("gdrive")

Mounted at gdrive


In [3]:
!unzip /content/gdrive/MyDrive/translation2019zh.zip

Archive:  /content/gdrive/MyDrive/translation2019zh.zip
  inflating: translation2019zh_train.json  
  inflating: translation2019zh_valid.json  


# read json

In [4]:
def is_chinese(strs):
    for _char in strs:
        if '\u4e00' <= _char <= '\u9fa5':
            return True
    return False

In [40]:
import json
import opencc
import re
train_data = []

x=1
chineseCc = opencc.OpenCC('s2hk.json')
with open("translation2019zh_train.json",newline="") as js:
  for i in js:
    data = json.loads(i)
    if data["english"].isascii() == False:
      continue
    if len(data["english"].strip()) > 25:
      continue
    if len(data["chinese"].strip()) > 10:
      continue
    data["english"] = data["english"].replace("\\","").strip()
    data["chinese"] = chineseCc.convert(data["chinese"]).strip()
    train_data.append(data)
    if(x==20000):
      break
    x+=1
  js.close()

In [42]:
train_data

[{'chinese': '選擇一個記錄員.', 'english': 'Choose a recorder.'},
 {'chinese': '側向運動。', 'english': 'Side-to-Side Movements.'},
 {'chinese': '大概華氏80度吧。', 'english': 'about like 80 degrees.'},
 {'chinese': '俺都是山東人。', 'english': 'We all are from Shandong.'},
 {'chinese': '傑克： 我喜歡滑冰。', 'english': 'Jake: I like to skate.'},
 {'chinese': '這種毛線很好織。', 'english': 'This wool knits up well.'},
 {'chinese': '石頭湯（故事去！）', 'english': 'Stone Soup Stories to Go!'},
 {'chinese': '一言為定。明天見！', 'english': 'Done. See you tomorrow.'},
 {'chinese': '脾氣真夠壞的！', 'english': 'What a terrible temper!'},
 {'chinese': '如果拉力很大', 'english': 'Well, if it was greater .'},
 {'chinese': '所有照片刻盤。', 'english': 'All photos dials.'},
 {'chinese': '一言難盡。説來話長。', 'english': "It's a long story."},
 {'chinese': '不要剪髮，只要修面。', 'english': 'No haircut, just a shave.'},
 {'chinese': '他的文體很明暢。', 'english': 'His style is very lucid .'},
 {'chinese': '事實真相已泄露。', 'english': 'The truth has leaked out.'},
 {'chinese': '他對我的畫評價很高。', 'english': 'He sw

In [43]:
txt_list = []

for data in train_data:
  strs = "%s\t%s"%(data["english"],data["chinese"])
  txt_list.append("%s\t%s"%(data["english"],data["chinese"]))

In [44]:
#wirte file to txt
with open("engChinese.txt","w",encoding="utf-8") as f:
  for data in txt_list:
    f.write(data)
    f.write("\n")

# Creating the model :Seq2Seq

In [45]:
from keras.models import Model
from keras.layers import Input, LSTM,Dense
import numpy as np

BATCH_SIZE = 64
EPOCHS = 200
LATENT_DIM = 256
NUM_SAMPLES = 10000
#traning setting

file_path = "engChinese.txt"

In [46]:
#Vetorize data
input_texts = []
target_texts = []
input_chars = set()
target_chars = set()

with open(file_path,"r",encoding="utf-8") as f:
  lines = f.read().split("\n") #each line is split by \n
  for line in lines[:min(NUM_SAMPLES,len(lines)-1)]: #num_spaces or line is larger,else will out of range
    input_text, target_text = line.split("\t") #left side of \t is end ,right side is chinese
    
    #each chinese texts need to add \t at the begining and \n at the end of the text
    target_text = "\t" + target_text +"\n"

    #add to input and target list
    input_texts.append(input_text)
    target_texts.append(target_text)

    #loopping the text to get all chars and make one-hot encoding later
    for char in input_text: #for english
      if char not in input_chars:
        input_chars.add(char)

    for char in target_text: # for chinese
      if char not in target_chars:
        target_chars.add(char)


In [47]:
len(target_chars)

3365

In [48]:
#handling the data to generate some required info
# sorting the chars 
input_chars = sorted(list(input_chars)) # according to Unicode
target_chars = sorted(list(target_chars)) # according to Unicode

#total chars of input and target
num_encoder_tokens = len(input_chars)
num_decoder_tokens = len(target_chars)

#the maxinums sentance of input and target
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

#log out the message for those info
print("Num of sample",len(input_texts))
print("Num of unique input token",num_encoder_tokens)
print("Num of unique output token",num_decoder_tokens)
print("Max seq length of inputs",max_encoder_seq_length)
print("Max seq length of output",max_decoder_seq_length)

Num of sample 10000
Num of unique input token 85
Num of unique output token 3365
Max seq length of inputs 25
Max seq length of output 12


In [ ]:
#Creating Encoder-LSTM

#make each chars has an index
input_token_index = dict([(char , i) for i ,char in enumerate(input_chars)]) # english token with index
target_token_index = dict([(char , i) for i ,char in enumerate(target_chars)]) # chinese token with index

#Need 3 array with 3-dimension(num_simple,max_seq_text,total_token)
#Encoder-input,Decoder-input,Decoder-output
encoder_input_data = np.zeros(
    (len(input_texts),max_encoder_seq_length,num_encoder_tokens), #shape-3d
    dtype=np.float32 #type float32
)

decoder_input_data = np.zeros(
    (len(input_texts),max_decoder_seq_length,num_decoder_tokens), #shape-3d
    dtype=np.float32 #type float32
)

decoder_output_data = np.zeros(
    (len(input_texts),max_decoder_seq_length,num_decoder_tokens), #shape-3d
    dtype=np.float32 #type float32
)

#zip eng sentance and chinese sentance(eng[0]Chinese[0],eng[1]Chinese[1]...etc)
for i,(input_text,target_text) in enumerate(zip(input_texts,target_texts)):
  #print(i,(input_text,target_text))
  for t,char in enumerate(input_text):
    #change the array of input data =>i:x,t:y,token:z to 1 = only set z-index to 1
    encoder_input_data[i,t,input_token_index[char]] = 1
  for t,char in enumerate(target_text):
    #chagne the array of target data
    #decoder_target_data is ahead decoder_input_data by one timestemp
    decoder_input_data[i,t,target_token_index[char]] = 1
    if t > 0:
      #only affect t > 0 character
      decoder_output_data[i,t-1,target_token_index[char]] = 1

# Encoder

In [ ]:
#Define input tensor
encoder_inputs = Input(shape=(None,num_encoder_tokens)) #Shape 1*num_encoder_tokens

#encoder LSTM with units = 256 and return the last state
encoder_lstm = LSTM((LATENT_DIM),return_state=True)

#output of encoder lstm tensor,return state history and state current
encoder_outputs,state_h,state_c = encoder_lstm(encoder_inputs)

#keep states only
encoder_states = [state_h,state_c]

# Decoder

In [ ]:
#need encoder states as decoder input
#define decoder lstm and tensor
decoder_inputs = Input(shape=(None,num_decoder_tokens)) #shape 1*num_decoder_tokens

#decoder_lstm ,set return everything
decoder_lstm = LSTM((LATENT_DIM),return_state=True,return_sequences=True)

#igore the internal state
decoder_outputs,_,_ = decoder_lstm(decoder_inputs)


In [ ]:
#creating the Dense for full connected layer
decoder_dense = Dense(num_decoder_tokens,activation="softmax") #num_decoder_tokens * 1

decoder_outputs = decoder_dense(decoder_outputs) #passing lstm output to decoder dense

In [ ]:
#Define the model
#input encoder input data and decoder input data and return decoder output data
model = Model([encoder_inputs,decoder_inputs],decoder_outputs)

#Run traning and complie the model
model.compile(optimizer="rmsprop",loss="categorical_crossentropy")

#fit data to model and training
model.fit([encoder_input_data,decoder_input_data],decoder_output_data,batch_size=BATCH_SIZE,epochs=EPOCHS,validation_split=0.2)

#save our model
model.save("seq2seq.h5")

Epoch 1/200
13/13 [==============================] - 39s 3s/step - loss: 2.2307 - val_loss: 2.1191
Epoch 2/200
13/13 [==============================] - 35s 3s/step - loss: 2.1114 - val_loss: 2.1015
Epoch 3/200
13/13 [==============================] - 35s 3s/step - loss: 2.0960 - val_loss: 2.0936
Epoch 4/200
13/13 [==============================] - 35s 3s/step - loss: 2.0877 - val_loss: 2.0870
Epoch 5/200
13/13 [==============================] - 35s 3s/step - loss: 2.0808 - val_loss: 2.0805
Epoch 6/200
13/13 [==============================] - 35s 3s/step - loss: 2.0761 - val_loss: 2.0783
Epoch 7/200
13/13 [==============================] - 35s 3s/step - loss: 2.0720 - val_loss: 2.0733
Epoch 8/200
13/13 [==============================] - 35s 3s/step - loss: 2.0683 - val_loss: 2.0724
Epoch 9/200
13/13 [==============================] - 35s 3s/step - loss: 2.0658 - val_loss: 2.0713
Epoch 10/200
13/13 [==============================] - 35s 3s/step - loss: 2.0642 - val_loss: 2.0707
Epoch 11/

In [ ]:
#compile the model
model.compile(optimizer="rmsprop",loss="categorical_crossentropy")

#model optimizsm
model.fit([encoder_input_dt,decoder_input_dt],decoder_output_dt,batch_size=64,epochs=100,validation_split=0.2)

#save model
model.save("seq2seq.h5")

Epoch 1/100
2/2 [==============================] - 39s 15s/step - loss: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 2/100
2/2 [==============================] - 33s 14s/step - loss: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 3/100
2/2 [==============================] - 33s 14s/step - loss: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 4/100
2/2 [==============================] - 32s 13s/step - loss: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 5/100
2/2 [==============================] - 32s 14s/step - loss: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 6/100
2/2 [==============================] - 32s 13s/step - loss: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 7/100
2/2 [==============================] - 32s 14s/step - loss: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 8/100
2/2 [==============================] - 32s 13s/step - loss: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 9/100
